In [6]:
import scanpy as sc
import pandas as pd
import numpy as np
import json
import geopandas as gpd
from shapely.geometry import Polygon, LineString
import matplotlib.pyplot as plt
from shapely import wkb
import csv

In [2]:
# df = pd.read_csv("./genes_raw/transcripts_z_43.csv")
# df

,Unnamed: 0,z,x,y,Cor_,Xdist_,Hraw,memdist,memdist2d,isnuc,icodesN,cid,gene,z_plane
0,3972,42.596617,368.060784,442.097685,0.439402,1.457996,152.109390,1.753848,2.358756,0.0,809.0,E1_1881.0,sox5,43.0
1,3973,42.757233,368.255092,442.123181,0.464502,1.437308,159.230977,1.860348,2.358756,0.0,810.0,E1_1881.0,sox5,43.0
2,3997,42.546078,438.179303,510.375109,0.671629,0.886841,245.393122,0.864000,2.173458,1.0,140.0,E1_1670.0,cldnd,43.0
3,3999,42.567083,449.786560,527.675836,0.586567,1.203291,286.048952,1.080000,1.858102,0.0,500.0,E1_1670.0,mab21l2,43.0
4,4004,42.605350,457.608467,472.263434,0.447917,1.493398,202.033241,1.620000,3.148714,1.0,833.0,E1_1206.0,syne2b,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099343,82867264,43.422396,1731.571320,1659.208675,0.850049,0.000000,216.713869,0.324000,1.069146,0.0,-11.0,E1_22963.0,nanos3,43.0
2099344,82867265,43.413569,1738.182648,1723.447109,0.770791,0.000000,227.155847,0.241495,0.341526,0.0,-11.0,E1_22854.0,nanos3,43.0
2099345,82867266,43.457354,1741.617585,1737.569500,0.824212,0.000000,166.051341,0.108000,0.540000,0.0,-11.0,E1_22854.0,nanos3,43.0
2099346,82867267,43.395899,1808.409333,1639.957134,0.849513,0.000000,232.359476,0.324000,1.404000,0.0,-11.0,E1_22451.0,nanos3,43.0


In [2]:
import os
import pandas as pd
import json
import gzip
import numpy as np
from tqdm import tqdm

# Create output directory if it doesn't exist
output_dir = './genes_optimized'
os.makedirs(output_dir, exist_ok=True)

# List of genes to process
genes_to_process_path = './gene_list.json'
if os.path.exists(genes_to_process_path):
    with open(genes_to_process_path) as f:
        genes_to_process = json.load(f)
else:
    genes_to_process = [
        "ackr3b",
        "acvr1ba",
        "add3b",
        "adka",
        "admp",
        "ahi1",
        "akap12b"
    ]

# Dictionary to store data for each gene
gene_data = {gene: {"layers": {}} for gene in genes_to_process}

# Process each layer
for layer in tqdm(range(0, 60), desc="Processing layers"):
    print(f"Processing layer {layer}...")
    
    # Read the CSV file for this layer
    layer_str = f"{layer:02d}" if layer < 10 else str(layer)
    csv_path = f"./genes_raw/transcripts_z_{layer_str}.csv"
    if not os.path.exists(csv_path):
        print(f"Warning: File {csv_path} not found, skipping.")
        continue
    
    df = pd.read_csv(csv_path)
    
    # Make sure 'gene' column exists (adjust column name if needed)
    if 'gene' not in df.columns:
        print(f"Error: 'gene' column not found in {csv_path}")
        print(f"Available columns: {df.columns.tolist()}")
        continue
    
    # Make sure 'x' and 'y' columns exist
    if 'x' not in df.columns or 'y' not in df.columns:
        print(f"Error: 'x' or 'y' columns not found in {csv_path}")
        print(f"Available columns: {df.columns.tolist()}")
        continue
    
    # Process each gene
    for gene in tqdm(genes_to_process, desc=f"Processing genes for layer {layer}"):
        # Filter data for this gene
        gene_df = df[df['gene'] == gene]
        
        if len(gene_df) == 0:
            # No data for this gene in this layer
            gene_data[gene]["layers"][str(layer)] = []
            continue
        
        # Extract x and y coordinates and flatten them
        coords = []
        for _, row in gene_df.iterrows():
            coords.append(int(row['x']) if row['x'].is_integer() else float(row['x']))
            coords.append(int(row['y']) if row['y'].is_integer() else float(row['y']))
        
        # Store the coordinates for this layer
        gene_data[gene]["layers"][str(layer)] = coords

# Save each gene's data to a compressed JSON file
for gene, data in tqdm(gene_data.items(), desc="Saving genes to disk"):
    # Skip genes with no data
    if all(len(coords) == 0 for coords in data["layers"].values()):
        print(f"Skipping {gene} - no data found")
        continue
    
    # Convert to JSON
    json_data = json.dumps(data)
    
    # Compress with gzip
    output_path = os.path.join(output_dir, f"{gene}.json.gz")
    with gzip.open(output_path, 'wt') as f:
        f.write(json_data)
    
    # Print file size
    file_size = os.path.getsize(output_path) / 1024  # KB
    print(f"Saved {gene}.json.gz ({file_size:.2f} KB)")

print("Processing complete!")

Processing layers:   0%|          | 0/60 [00:00<?, ?it/s]

Processing layer 0...


Processing genes for layer 0: 100%|██████████| 495/495 [00:00<00:00, 18091.34it/s]


Processing layer 1...


Processing genes for layer 1: 100%|██████████| 495/495 [00:00<00:00, 15957.36it/s]


...

Saved znf576.2.json.gz (2855.13 KB)


Saving genes to disk: 100%|█████████▉| 494/495 [03:50<00:00,  1.91it/s]

Saved znfl2a.json.gz (5157.00 KB)


Saving genes to disk: 100%|██████████| 495/495 [03:50<00:00,  2.15it/s]

Saved zswim5.json.gz (3314.35 KB)
Processing complete!
